# RDDs

In [ ]:
from pyspark import SparkContext
sc = SparkContext(master = "local", appName = "Transformaciones sobre un RDD") 

Carga de archivo paises

In [ ]:
path = "../proyecto/"
equiposOlimpicosRDD = sc.textFile(path+"paises.csv").map(lambda line : line.split(","))

In [ ]:
equiposOlimpicosRDD.take(10)

Determinamos la cantidad de siglas para los equipos olímpicos existen

In [ ]:
equiposOlimpicosRDD.map(lambda x : (x[2])).distinct().count()

Agrupamos datos para poder determinar cuantos equipos posee un pais

In [ ]:
equiposOlimpicosRDD.map(lambda x : (x[2], x[1] )).groupByKey().mapValues(list).collect()


In [ ]:
equiposOlimpicosRDD.map(lambda x :  (x[2], x[1] )).groupByKey().mapValues(len).collect()


In [ ]:
equiposArgentinos = equiposOlimpicosRDD.filter(lambda l : "ARG" in l)
equiposArgentinos.collect()  

In [ ]:
deportistaOlimpicoRDD = sc.textFile(path+"deportista.csv").map(lambda line : line.split(","))
deportistaOlimpico2RDD = sc.textFile(path+"deportista2.csv").map(lambda line : line.split(","))

In [ ]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.union(deportistaOlimpico2RDD)

# Accciones sobre RDDs

Existen tres formas básicas para poder selecionar datos de un RDD

Collect, una cuarta forma, no es recomendable de ser utilizada ya que esta puede generar una sobrecarga en el driver del programa 

In [ ]:
deportistaOlimpicoRDD.map(lambda x :[x[-1], x[:-1]]).join(equiposOlimpicosRDD.map(lambda x : [x[0],x[2]])).take(6)

In [ ]:
deportistaOlimpicoRDD.map(lambda x :[x[-1], x[:-1]]).join(equiposOlimpicosRDD.map(lambda x : [x[0],x[2]])).top(5)

In [ ]:
deportistaOlimpicoRDD.map(lambda x :[x[-1], x[:-1]]).join(equiposOlimpicosRDD.map(lambda x : [x[0],x[2]])).takeSample(False,4,10)

## Importancia de countAprox

Debido a la cantidad de datos no es recomendable hacer operaciones tipo count.
Por lo que countAprox es la solución mas viable cuando solo queremos darnos una idea de cuantos datos podemos leer durante un tiempo determnado

In [ ]:
deportistaOlimpicoRDD.map(lambda x :[x[-1], x[:-1]]).join(equiposOlimpicosRDD.map(lambda x : [x[0],x[2]])).count()

In [ ]:
deportistaOlimpicoRDD.map(lambda x :[x[-1], x[:-1]]).join(equiposOlimpicosRDD.map(lambda x : [x[0],x[2]])).countApprox(20)

## Acciones de modificacion

In [ ]:
equipoDeportista = deportistaOlimpicoRDD.map(lambda x :[x[-1], x[:-1]]).join(equiposOlimpicosRDD.map(lambda x : [x[0],x[2]]))

In [ ]:
equipoDeportista.take(1)


In [ ]:
equipoDeportista.map(lambda x : (x[1][0][0],x[1][0][1:],x[1][1]) )

In [ ]:
resultado = sc.textFile(path+"athlete_events.csv").map(lambda line : line.split(","))

In [ ]:
resultadoGanador = resultado.filter(lambda l : 'NA' not in l[1])
resultadoGanador = resultadoGanador.map(lambda l : [l[2],l[1]])

In [ ]:
resultadoGanador.take(15)

In [ ]:
jugadoresMedalla =  equipoDeportista.join(resultadoGanador)
jugadoresMedalla.take(1)

Agrupamos las medallas respecto a la sigla del pais jugador

In [ ]:
d = {'Gold':7, 'Silver':5, 'Bronze':4}
paisesMedallas = jugadoresMedalla.map(lambda x : (x[1][0][-1],d[x[1][1]]) )
paisesMedallas.takeSample(False,10)

Obtenermos los valores de los puntuajes históricos de los paises jugadores

In [ ]:
from operator import add
conclusion = paisesMedallas.reduceByKey((add)).sortBy(lambda x : x[1],ascending=False)
conclusion.take(10)

Estadística básica sobre los RDDs

In [ ]:
conclusion.map(lambda l : l[1]).stats()

In [ ]:
conclusion.map(lambda l : l [1]).mean()

In [ ]:
conclusion.map(lambda l : l [1]).sum()

In [ ]:
conclusion.map(lambda l : l [1]).histogram(10)